## Societal Acceptance of the LGBTQ+ Community: A Demographic and Attitudinal Analysis


In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("Packages imported successfully")

Packages imported successfully


## Data Import

In [6]:
data = pd.read_excel("/Users/mari/Desktop/Github/societalacceptanceoflgbtqcommunity/societal_acceptance.xlsx")
df = pd.DataFrame(data)

## EDA - Exploratory Data Analysis

In [9]:
# Display the size of the dataframe
df.shape

(177, 20)

In [10]:
# Display the first few rows of the dataset
df.head(5)

,#,How old are you? Please see the appropriate age range.,How would you describe your ethnicity?,How do you identify?,How would you describe your sexual orientation?,Other,What is the highest level of school you have completed or the highest degree you have received?,Do you identify with any of the following religions?,Other.1,How would you describe your political viewpoints?,Do you identify as a member of the LGBTQ+ community?,Do you know anyone personally who identifies as LGBTQ+?,How frequently do you interact with members of the LGBTQ+ community?,"On a scale of 1-10, how would you rate your overall understanding of LGBTQ+ issues?",What is your primary source of information about LGBTQ+ issues? (Please choose the best option from this list),"On a scale of 1-10, how important do you believe it is to educate the public about LGBTQ+ issues?","On a scale of 1-10, how would you rate your overall acceptance of the LGBTQ+ community?",Do you believe that LGBTQ+ individuals should have the same rights as non-LGBTQ+ individuals?,Have your views on the LGBTQ+ community changed over the last few years?,How would you react if a friend or family member came out as LGBTQ+?
0,ztfw5stwq4m3xtztfyewlpxbl6frb80s,45-54,Black or African-American,Female,Straight,NaN,Some college but no degree,Christianity,NaN,Moderate,0,1,Daily,6,Personal experiences or encounters,8,10,1,1,I would support and accept them unconditionally.
1,rzna89fakp14yfqrwyd7vdrzna89fs0i,35-44,Asian,Male,Straight,NaN,Bachelor degree,Catholicism,NaN,Slightly liberal,1,1,Daily,6,Personal experiences or encounters,9,7,1,1,I would support and accept them unconditionally.
2,66rxhe42rn2tv966rxhtghua7iyc0li3,25-34,White,Prefer not to disclose,Straight,NaN,Bachelor degree,No religion,NaN,Slightly liberal,0,1,Monthly,6,Personal experiences or encounters,6,8,1,0,I would express my love and continue to treat ...
3,f4xo4trvwusznv04fu8f4x1nd9hmpiqa,25-34,White,Male,Straight,NaN,Some college but no degree,No religion,NaN,Slightly liberal,0,1,Weekly,8,Personal experiences or encounters,7,8,1,0,I would express my love and continue to treat ...
4,kjesldgd5gvzfwdg54u8r6kjeslm7i43,25-34,Black or African-American,Male,Straight,NaN,Some college but no degree,Christianity,NaN,Very liberal,0,1,Daily,7,Friends or family members,9,10,1,0,I would support and accept them unconditionally.
